# Vertex Pipelines with R - Introduction 

An introductory example of using R with Vertex Pipelines 

### Install additional packages



On colab, authenticate first:

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

Then, install the libraries.

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

We will be using [Vertex SDK for Python](https://cloud.google.com/vertex-ai/docs/start/client-libraries#python) to interact with Vertex AI services. The high-level aiplatform library is designed to simplify common data science workflows by using wrapper classes and opinionated defaults.

#### Install Vertex SDK for Python

In [ ]:
!pip -q install {USER_FLAG} --upgrade kfp
!pip -q install {USER_FLAG} --upgrade google-cloud-pipeline-components 
!pip -q install {USER_FLAG} --upgrade google-cloud-aiplatform

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Check the versions of the packages you installed.  The KFP SDK version should be >=1.6.

In [ ]:
! python3 -c "import kfp; print('kfp version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

## Before you begin

This notebook does not require a GPU runtime.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os
# Get your Google Cloud project ID from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null

try:
    PROJECT_ID = shell_output[0]
except IndexError:
    PROJECT_ID = None

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "python-docs-samples-tests"  # @param {type:"string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

def get_timestamp():
    return datetime.now().strftime("%Y%m%d%H%M%S")

TIMESTAMP = get_timestamp()
print(f"TIMESTAMP = {TIMESTAMP}")

### Create a Cloud Storage bucket as necessary



In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "-aip-r-test" + TIMESTAMP
BUCKET_NAME

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Import libraries and define constants



Define some constants. See the "Before you begin" section of the Managed Pipelines User Guide for information on creating your API key.


In [ ]:
APP_NAME = "r-test"

Do some imports:

In [ ]:
import json
from typing import NamedTuple, List

from google_cloud_pipeline_components import aiplatform as aip_components
from google_cloud_pipeline_components.v1.custom_job import CustomTrainingJobOp
        
from google_cloud_pipeline_components.types import artifact_types
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs

from kfp.v2 import compiler
from kfp.v2 import dsl
from kfp.v2 import components

In [ ]:
! mkdir ./src

#### Enable Artifact Registry API

First, you must enable the Artifact Registry API service for your project.

Learn more about [Enabling service](https://cloud.google.com/artifact-registry/docs/enable-service).

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

#### Create a private Docker repository

Your first step is to create your own Docker repository in Google Artifact Registry.

1. Run the `gcloud artifacts repositories create` command to create a new Docker repository with your region with the description "docker repository".

2. Run the `gcloud artifacts repositories list` command to verify that your repository was created.

In [ ]:
PRIVATE_REPO = "r-test-repo"

In [ ]:
! gcloud artifacts repositories create $PRIVATE_REPO --repository-format=docker \
    --location=$REGION \
    --description="Docker repository for R testing"

In [ ]:
! gcloud artifacts repositories list

In [ ]:
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet

## Create script 

for printing / testing

In [ ]:
%%writefile src/print.R
#!/usr/bin/env Rscript
# printing test

print("This is printed using the print function")

message("This is printed using the message function")

cat("This is printed using the cat function")

write("This is printed using the write function", stdout())

# view environment vars
Sys.getenv()

## Build and push container image

### Create Dockerfile

The docker file for your custom container is built on top of the Deep Learning container -- the same container that is also used for Vertex AI Workbench. In addition, you add two R scripts for model training and serving, respectively.

In [ ]:
IMAGE_NAME = "r-test-image"  # @param {type:"string"}
IMAGE_TAG = "latest"  # @param {type:"string"}
IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE_NAME}:{IMAGE_TAG}"
IMAGE_URI

In [ ]:
%%writefile ./src/Dockerfile

FROM gcr.io/deeplearning-platform-release/r-cpu.4-1:latest

WORKDIR /root

COPY print.R /root/print.R

RUN apt-get update
RUN apt-get install gfortran -yy

### Build the Docker container

Next, you build the Docker container image on Cloud Build -- the serverless CI/CD platform.

*Note:* Building the Docker container image may take 10 to 15 minutes.

In [ ]:
! gcloud builds submit --region=$REGION --tag=$IMAGE_URI --timeout=1h ./src --async

## Define the pipeline components



## Define Configuration

In [ ]:
MODEL_NAME = APP_NAME
MODEL_DISPLAY_NAME = f"{MODEL_NAME}"

PIPELINE_NAME = f"{APP_NAME}-pipeline"
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/{MODEL_NAME}"
GCS_STAGING = f"{BUCKET_NAME}/pipeline_root/{MODEL_NAME}"

IMAGE_URI # defined above

PIPELINE_JSON_SPEC_PATH = './src/r-test-pipeline-spec.json'

WORKING_DIR = f"{PIPELINE_ROOT}/{TIMESTAMP}"
WORKING_DIR

## Define Pipeline


* Example - Pipeline - https://github.com/JowGarrido/kfp-r-models/blob/main/pipeline_definition.ipynb 
* SDK Vertex - https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob
* API Vertex - https://cloud.google.com/vertex-ai/docs/reference/rest/v1/CustomJobSpec
* Pipeline component - https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.26/google_cloud_pipeline_components.v1.custom_job.html


In [ ]:
@dsl.pipeline(
    name = PIPELINE_NAME, 
    pipeline_root = PIPELINE_ROOT)
def r_pipeline():
    print_task = (
        CustomTrainingJobOp(
            project = PROJECT_ID,
            location = REGION,
            display_name = "Run R script",
            base_output_directory = WORKING_DIR,
            worker_pool_specs=[
                {
                    "containerSpec": {
                        "imageUri": IMAGE_URI,
                        "command": ["Rscript", "print.R"]
                    },
                    "replicaCount": "1",
                    "machineSpec": {
                        "machineType": "n1-standard-16"
                    },
                }
            ],
        )
        .set_display_name("Run R script")
        .set_caching_options(False)
    )

In [ ]:
compiler.Compiler().compile(pipeline_func = r_pipeline, 
                            package_path = PIPELINE_JSON_SPEC_PATH)

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

In [ ]:
job = pipeline_jobs.PipelineJob(
    display_name = 'r_print_test',
    template_path = PIPELINE_JSON_SPEC_PATH,
    pipeline_root = PIPELINE_ROOT,
    enable_caching = False
)

job.submit()